In [1]:
# Libraries for data loading, data manipulation and data visulisation
import pandas as pd  # Data processing and manipulation 
import matplotlib.pyplot as plt  # Data visualization
import seaborn as sns  # Statistical data visualization
import numpy as np  # Numerical operations and linear algebra 

# Libraries for data preparation and model building
from scipy.stats import norm  # Statistical functions
from sklearn.preprocessing import StandardScaler  # Data preprocessing
import warnings  # Warning handling
warnings.filterwarnings('ignore')  # Ignore warnings

### Loading Original Dataset

In [2]:
# load the data, used the pandas read_csv() to import our dataset 
data = pd.read_excel('DiabetesDB.xlsx', sheet_name=None)
#df.head() #Preview the dataset

In [3]:
# Cocantenation
df_Original = pd.concat(data.values(), ignore_index=True)

In [4]:
# load the data, used the pandas read_csv() to import our dataset 
df_Extra = pd.read_csv('DiabetesDB_Extra.csv')
#df.head() #Preview the dataset

In [5]:
df_Extra.columns

Index(['Diabetes_binary', 'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker',
       'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education',
       'Income'],
      dtype='object')

In [6]:
# Step 1: Create 'Full_Name' column
df_Original['FULL_NAME'] = df_Original['NAME '].fillna('') + ' ' + df_Original['SURNAME'].fillna('')

# Step 2: Reorder columns to place 'Full_Name' first
columns = ['FULL_NAME'] + [col for col in df_Original.columns if col not in ['FULL_NAME']]
df_Original = df_Original[columns]


In [7]:
df_Original = df_Original.drop(['NAME ', 'SURNAME', 'PHONE ', 'EMAIL','STATE OF ORIGIN', 'NAME', 'PHONE CALL ', 'CONTACT', 'RELATIONSHIP', 'FILE  LINK'], axis=1)

#### COde for BMI

In [8]:
import re

def clean_and_convert_to_meters(value):
    # Remove non-numeric characters and decimal points
    cleaned_value = re.sub(r'\D', '', str(value))
    
    if cleaned_value:
        # Convert to float and treat it as centimeters
        height_in_cm = float(cleaned_value)
        # Convert cm to meters
        return height_in_cm / 100
    else:
        return None  # Return None if value is empty or non-numeric

# Apply the function to the 'height' column
df_Original['HEIGHT'] = df_Original['HEIGHT'].apply(clean_and_convert_to_meters)


# Apply the function to the 'height(cm)' column
df_Original['HEIGHT (cm)'] = df_Original['HEIGHT (cm)'].apply(clean_and_convert_to_meters)



In [9]:
# Create a new 'height' column
df_Original['HEIGHT'] = df_Original['HEIGHT'].fillna(df_Original['HEIGHT (cm)'])

# Drop the 'height (cm)' column now that data is merged
df_Original.drop(columns=['HEIGHT (cm)'], inplace=True)

# Calculate the mean height from available values and round to 2 decimal places
mean_height = round(df_Original['HEIGHT'].mean(), 2)

# Fill missing values in 'height' with the rounded mean
df_Original['HEIGHT'].fillna(mean_height, inplace=True)

In [10]:
# Function to remove non-numeric characters from weight column
def clean_weight(value):
    # Remove all non-numeric characters, keep only digits and decimal points
    cleaned_value = re.sub(r'[^0-9.]', '', str(value))
    return float(cleaned_value) if cleaned_value else None

# Apply the function to the 'weight' column
df_Original['WEIGHT'] = df_Original['WEIGHT'].apply(clean_weight)

In [18]:
# Calculate BMI and add it as a new column in the DataFrame
df_Original['BMI'] = df_Original['WEIGHT'] / (df_Original['HEIGHT'] ** 2)

# Round BMI to 2 decimal places
df_Original['BMI'] = df_Original['BMI'].round(0)
#df_Original['BMI'] = df_Original['BMI']

In [19]:
df_Original.head()

,FULL_NAME,GENDER,YOB,GLUCOSE,BLOOD P.,BMI,HEIGHT,WEIGHT
0,ALICE JOSEPH,NaN,70 YEARS,8.2MMOL,104/67MMHG,9.0,2.38,48.3
1,AMOS KPAKACHI,NaN,60 YRS,10.1MMOL,127/74MMHG,11.0,2.38,62.7
2,AYUBA BARDE,NaN,50 YRS,7.3MM0L,90/60MMHG,12.0,2.38,68.9
3,AYUBA GWAZAWA,NaN,1982,6.9MMOL,123/79MMGH,11.0,2.38,59.8
4,BEAUTY SUNDAY,NaN,1998,4.4MMOL,104/80MMHG,13.0,2.38,72.4


### Code to Clean Gender Column

In [20]:
# Fill missing values in 'Gender' with 'Unknown'
df_Original['GENDER'].fillna('Unknown', inplace=True)

# Define mapping for 'Male' and 'Female', with 'Unknown' as a separate category if needed
gender_mapping = {'MALE': 0, 'FEMALE': 1, 'Unknown': 2}
df_Original['GENDER'] = df_Original['GENDER'].map(gender_mapping)


In [21]:
# Rename 'GENDER' to 'Sex' in the old dataset
df_Original.rename(columns={'GENDER': 'Sex'}, inplace=True)


### Code to Clean Age Column

In [22]:
from datetime import datetime

# Sample data with 'YOB' column (containing both year of birth and age)
#data = {
    #'YOB ': ['1995', '74 years', '1985', 'N/A', '30', '2000', '45 years', '1982', '56', 'unknown']
#}
#df = pd.DataFrame(data)

# Get the current year
current_year = datetime.now().year

# Function to clean and convert the YOB data
def clean_yob(value):
    if isinstance(value, str):  # Check if the value is a string
        # Try to extract the numeric part from the string
        digits = ''.join([ch for ch in value if ch.isdigit()])
        if len(digits) == 4:  # If it's a 4-digit number, assume it's a year of birth
            return int(digits)
        elif len(digits) > 0 and len(digits) < 3:  # If it's age (usually 2 digits), calculate YOB
            try:
                return current_year - int(digits)
            except ValueError:
                return None
    elif isinstance(value, (int, float)):  # If the value is already numeric
        if value < current_year:  # If the number is less than the current year, assume it's a year of birth
            return int(value)
    return None  # For any invalid cases (e.g., 'N/A', 'unknown', etc.)

# Apply the function to clean the 'YOB' column
df_Original['YOB_cleaned'] = df_Original['YOB '].apply(clean_yob)

In [23]:
# Assuming `df` is the DataFrame with the cleaned 'YOB_cleaned' column
current_year = datetime.now().year

# Calculate age by subtracting the year of birth from the current year
df_Original['Age'] = current_year - df_Original['YOB_cleaned']

# Convert the 'Age' column to integer, handling missing values
df_Original['Age'] = df_Original['Age'].fillna(df_Original['Age'].median()).astype(int)



### Code to Clean Glucose Column

In [24]:
# First, clean the 'glucose' column
df_Original['GLUCOSE '] = df_Original['GLUCOSE '].str.replace('MMOL', '').str.strip()  # Remove 'MMOL' and any extra spaces
df_Original['GLUCOSE '] = pd.to_numeric(df_Original['GLUCOSE '], errors='coerce')      # Convert to numeric, set invalid parsing as NaN

# Fill missing values with the mean (you could also use median or another method)
df_Original['GLUCOSE '].fillna(df_Original['GLUCOSE '].mean(), inplace=True)

# Round the 'glucose' values to 2 decimal places
df_Original['GLUCOSE '] = df_Original['GLUCOSE '].round(2)

# Display the cleaned and rounded 'glucose' column
print(df_Original['GLUCOSE '])

0       8.20
1      10.10
2       6.97
3       6.90
4       4.40
       ...  
146     8.20
147     4.80
148     8.30
149     5.20
150     5.80
Name: GLUCOSE , Length: 151, dtype: float64


### Code to Clean Blood Pressure Column

In [25]:
# Remove 'MMHG' and any extra spaces
df_Original['BLOOD P.'] = df_Original['BLOOD P.'].str.replace('MMHG', '').str.strip()

# Split the blood pressure values into systolic and diastolic columns
df_Original[['Systolic', 'Diastolic']] = df_Original['BLOOD P.'].str.split('/', expand=True)

# Convert both columns to numeric
df_Original['Systolic'] = pd.to_numeric(df_Original['Systolic'], errors='coerce')
df_Original['Diastolic'] = pd.to_numeric(df_Original['Diastolic'], errors='coerce')

# Optional: Round the values to 2 decimal places (if necessary)
df_Original['Systolic'] = df_Original['Systolic'].round(2)
df_Original['Diastolic'] = df_Original['Diastolic'].round(2)

# Optionally, you can drop the original 'BLOOD P.' column
df_Original.drop(columns=['BLOOD P.'], inplace=True)

# Display the cleaned data
print(df_Original[['Systolic', 'Diastolic']])


     Systolic  Diastolic
0       104.0       67.0
1       127.0       74.0
2        90.0       60.0
3       123.0        NaN
4       104.0       80.0
..        ...        ...
146      85.0       64.0
147     113.0       70.0
148     115.0       94.0
149     116.0       75.0
150     105.0       66.0

[151 rows x 2 columns]


### FINAL ORDERED COLUMNS

In [26]:
# Remove unnecessary columns
#df_Original.drop(columns=['YOB ', 'YOB_cleaned'], inplace=True)

# Define the new column order
new_order = ['FULL_NAME', 'Sex', 'Age', 'GLUCOSE ', 'Systolic', 'Diastolic', 'HEIGHT', 'WEIGHT', 'BMI']

# Rearrange the columns
df_Original = df_Original[new_order]

# Display the updated DataFrame
print(df_Original.head())


        FULL_NAME  Sex  Age  GLUCOSE   Systolic  Diastolic  HEIGHT  WEIGHT  \
0    ALICE JOSEPH  2.0   70      8.20     104.0       67.0    2.38    48.3   
1   AMOS KPAKACHI  2.0   60     10.10     127.0       74.0    2.38    62.7   
2     AYUBA BARDE  2.0   50      6.97      90.0       60.0    2.38    68.9   
3   AYUBA GWAZAWA  2.0   42      6.90     123.0        NaN    2.38    59.8   
4  BEAUTY  SUNDAY  2.0   26      4.40     104.0       80.0    2.38    72.4   

    BMI  
0   9.0  
1  11.0  
2  12.0  
3  11.0  
4  13.0  


### Creating column for Diabetes Status

In [27]:
df_Original['Diabetes_binary'] = ((df_Original['BMI'] > 30) | 
                  (df_Original['Systolic'] > 140) | 
                  (df_Original['Diastolic'] > 90) | 
                  (df_Original['Age'] > 45)).astype(int)
df_Original['Diabetes_binary'] = df_Original['Diabetes_binary'].astype(int)  # Convert to 0 (no diabetes) and 1 (diabetes)

In [28]:
df_Original.columns

Index(['FULL_NAME', 'Sex', 'Age', 'GLUCOSE ', 'Systolic', 'Diastolic',
       'HEIGHT', 'WEIGHT', 'BMI', 'Diabetes_binary'],
      dtype='object')

### CLEANING AND PROCESSING THE NEW DATASET

In [29]:
df_Extra.columns

Index(['Diabetes_binary', 'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker',
       'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education',
       'Income'],
      dtype='object')

In [30]:
relevant_columns = [
    'Diabetes_binary', 'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker',
    'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump',
    'HeartDiseaseorAttack', 'Age', 'Sex'
]
df_new = df_Extra[relevant_columns]


In [31]:
df_new.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,HeartDiseaseorAttack,Age,Sex
0,0.0,1.0,0.0,1.0,26.0,0.0,1.0,0.0,1.0,0.0,0.0,4.0,1.0
1,0.0,1.0,1.0,1.0,26.0,1.0,0.0,1.0,0.0,0.0,0.0,12.0,1.0
2,0.0,0.0,0.0,1.0,26.0,0.0,1.0,1.0,1.0,0.0,0.0,13.0,1.0
3,0.0,1.0,1.0,1.0,28.0,1.0,1.0,1.0,1.0,0.0,0.0,11.0,1.0
4,0.0,0.0,0.0,1.0,29.0,1.0,1.0,1.0,1.0,0.0,0.0,8.0,0.0


### Create New Columns in the Old Dataset

In [32]:
# Identify missing columns
missing_columns = ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 
                   'PhysActivity', 'Fruits', 'Veggies', 
                   'HvyAlcoholConsump', 'HeartDiseaseorAttack']

# Create missing columns in the old dataset
for col in missing_columns:
    if col not in df_Original.columns:
        df_Original[col] = np.nan  # Initialize with NaN

# Populate 'Sex' from 'GENDER'
# Assume 'Male' -> 1, 'Female' -> 0 (map accordingly)
#df_Original['Sex'] = df_Original['GENDER'].map({'Male': 1, 'Female': 0}).fillna(np.nan)

# Preview the updated old dataset
print(df_Original.head())


        FULL_NAME  Sex  Age  GLUCOSE   Systolic  Diastolic  HEIGHT  WEIGHT  \
0    ALICE JOSEPH  2.0   70      8.20     104.0       67.0    2.38    48.3   
1   AMOS KPAKACHI  2.0   60     10.10     127.0       74.0    2.38    62.7   
2     AYUBA BARDE  2.0   50      6.97      90.0       60.0    2.38    68.9   
3   AYUBA GWAZAWA  2.0   42      6.90     123.0        NaN    2.38    59.8   
4  BEAUTY  SUNDAY  2.0   26      4.40     104.0       80.0    2.38    72.4   

    BMI  Diabetes_binary  HighBP  HighChol  CholCheck  Smoker  PhysActivity  \
0   9.0                1     NaN       NaN        NaN     NaN           NaN   
1  11.0                1     NaN       NaN        NaN     NaN           NaN   
2  12.0                1     NaN       NaN        NaN     NaN           NaN   
3  11.0                0     NaN       NaN        NaN     NaN           NaN   
4  13.0                0     NaN       NaN        NaN     NaN           NaN   

   Fruits  Veggies  HvyAlcoholConsump  HeartDiseaseorAtt

In [33]:
df_Original.columns

Index(['FULL_NAME', 'Sex', 'Age', 'GLUCOSE ', 'Systolic', 'Diastolic',
       'HEIGHT', 'WEIGHT', 'BMI', 'Diabetes_binary', 'HighBP', 'HighChol',
       'CholCheck', 'Smoker', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'HeartDiseaseorAttack'],
      dtype='object')

In [34]:
# Final list of columns in the new dataset
new_dataset_columns = [
    'FULL_NAME', 'Sex', 'Age', 'GLUCOSE ', 'Systolic', 'Diastolic', 
    'HEIGHT', 'WEIGHT', 'BMI', 'Diabetes_binary', 
    'HighBP', 'HighChol', 'CholCheck', 'Smoker', 'PhysActivity', 
    'Fruits', 'Veggies', 'HvyAlcoholConsump', 'HeartDiseaseorAttack'
]

# Drop columns in the old dataset not present in the new dataset
df_Original_cleaned = df_Original[[col for col in df_Original.columns if col in df_new]]


In [35]:
df_Original_cleaned.columns

Index(['Sex', 'Age', 'BMI', 'Diabetes_binary', 'HighBP', 'HighChol',
       'CholCheck', 'Smoker', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'HeartDiseaseorAttack'],
      dtype='object')

In [36]:
finl_dataset_columns = ['Sex', 'Age', 'BMI', 'Diabetes_binary', 'HighBP', 'HighChol',
       'CholCheck', 'Smoker', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'HeartDiseaseorAttack']

In [37]:
# Reorder old dataset columns to match the new dataset
df_Original_cleaned = df_Original_cleaned[finl_dataset_columns]

# Reorder new dataset columns to ensure they match (optional, for safety)
df_new_cleaned = df_new[finl_dataset_columns]


In [38]:
# Final Preview
print("Old Dataset Columns:", df_Original_cleaned.columns)
print("New Dataset Columns:", df_new_cleaned.columns)

Old Dataset Columns: Index(['Sex', 'Age', 'BMI', 'Diabetes_binary', 'HighBP', 'HighChol',
       'CholCheck', 'Smoker', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'HeartDiseaseorAttack'],
      dtype='object')
New Dataset Columns: Index(['Sex', 'Age', 'BMI', 'Diabetes_binary', 'HighBP', 'HighChol',
       'CholCheck', 'Smoker', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'HeartDiseaseorAttack'],
      dtype='object')


In [53]:
# Merge the datasets
merged_df = pd.concat([df_Original_cleaned, df_new_cleaned], ignore_index=True)

# Preview merged data
print(merged_df.head(500))


     Sex   Age   BMI  Diabetes_binary  HighBP  HighChol  CholCheck  Smoker  \
0    2.0  70.0   9.0              1.0     NaN       NaN        NaN     NaN   
1    2.0  60.0  11.0              1.0     NaN       NaN        NaN     NaN   
2    2.0  50.0  12.0              1.0     NaN       NaN        NaN     NaN   
3    2.0  42.0  11.0              0.0     NaN       NaN        NaN     NaN   
4    2.0  26.0  13.0              0.0     NaN       NaN        NaN     NaN   
..   ...   ...   ...              ...     ...       ...        ...     ...   
495  0.0   3.0  25.0              0.0     0.0       0.0        1.0     0.0   
496  1.0   4.0  35.0              0.0     0.0       0.0        1.0     1.0   
497  0.0  13.0  28.0              0.0     1.0       1.0        1.0     0.0   
498  0.0   7.0  25.0              0.0     0.0       0.0        1.0     0.0   
499  1.0  12.0  22.0              0.0     0.0       0.0        1.0     1.0   

     PhysActivity  Fruits  Veggies  HvyAlcoholConsump  HeartDis

In [56]:
# Fill null values in each column with the mode
for column in merged_df.columns:
    if merged_df[column].isnull().sum() > 0:  # Check if there are null values
        mode_value = merged_df[column].mode()[0]  # Calculate the mode
        merged_df[column].fillna(mode_value, inplace=True)  # Fill nulls with the mode

# Verify that there are no missing values
print(merged_df.isnull().sum())



Sex                     0
Age                     0
BMI                     0
Diabetes_binary         0
HighBP                  0
HighChol                0
CholCheck               0
Smoker                  0
PhysActivity            0
Fruits                  0
Veggies                 0
HvyAlcoholConsump       0
HeartDiseaseorAttack    0
dtype: int64


In [57]:
merged_df.head()

,Sex,Age,BMI,Diabetes_binary,HighBP,HighChol,CholCheck,Smoker,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,HeartDiseaseorAttack
0,2.0,70.0,9.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
1,2.0,60.0,11.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
2,2.0,50.0,12.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
3,2.0,42.0,11.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
4,2.0,26.0,13.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0


In [58]:
# Reorder columns to place 'Diabetes_binary' at the end
columns = [col for col in merged_df.columns if col != 'Diabetes_binary'] + ['Diabetes_binary']
merged_df = merged_df[columns]

# Verify the new column order
print(merged_df.columns)


Index(['Sex', 'Age', 'BMI', 'HighBP', 'HighChol', 'CholCheck', 'Smoker',
       'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump',
       'HeartDiseaseorAttack', 'Diabetes_binary'],
      dtype='object')


In [61]:
# Export the merged dataset to a CSV file
merged_df.to_csv('merged_diabetes_dataset.csv', index=False)

print("Dataset exported successfully to 'merged_diabetes_data.csv'")


Dataset exported successfully to 'merged_diabetes_data.csv'
